# 1. Librerías

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from sklearn import metrics
import shap
#from utils.Validator import *
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

c:\GIT_CUMP4\epic\.venv\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\GIT_CUMP4\epic\.venv\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks, max_swap_size=100, 

# 2. Data

In [2]:
dat= pd.read_csv(r"../data/modeling/03_train_ppjj.csv",sep="|")
dat.head()

,PERIODO,CODCLAVECIC,EDAD,TIPPER,ANTIGUEDAD,CODACTECONOMICA,DESACTECONOMICA,FLG_ACTECO_NODEF,FLGNP,CTDNP,FLGLSB,CTDLSB,FLGARCHIVONEGATIVO,DESTIPMOTIVONEGATIVO,CTD_NP_LSB,CTDEVAL,CODSUBSEGMENTO,DESSUBSEGMENTO,CODSEGMENTO,DESSEGMENTO,TIPDIR,CODUBIGEO,CODDISTRITO,DESCODDISTRITO,CODPROVINCIA,DESCODPROVINCIA,CODDEPARTAMENTO,DESCODDEPARTAMENTO,CODPAISNACIONALIDAD,DESCODPAISNACIONALIDAD,FLGNACIONALIDAD,MTO_INGRESOS_MES,CTD_INGRESOS_MES,MEDIA_INGRESOS,DESV_INGRESOS,FLG_PERFIL_INGRESOS_3DS,MAX_CTDMTOSREDONDOS,SUM_MTOSREDONDOS,CTDMAXIMA,MTO_MAXIMOSREPETIDOS,MTOMAXDEMAXREPETIDOS,MTO_CONOTROSPROXIMOS,CTD_CONOTROSPROXIMOS
0,202309,9304309,12,E,147,500.0000,"PESCA,CRIADEROS Y ACTIVIDS.RELACIONADAS",0,0,0,0,0,0,NaN,0,0,Q1N,PEQ. EMPRESA NORMAL,PQ,PEQ. EMPRESA,D,160101,1400,IQUITOS-LORETO,N3,MAYNAS,32.0000,LORETO,PER,PERU,0,52345.0000,25,16017.0300,7829.8100,1,4,52345.0000,4,1800.0000,450.0000,2275.0000,5
1,202309,2763563,7,E,291,6420.0000,TELECOMUNICACION,0,1,5,1,2,0,NaN,7,24,CDN,COMERCIAL SUBSEGMENTO D NORMAL,CC,COMERCIAL,D,150115,1310,LA VICTORIA-LIMA,M7,LIMA,30.0000,LIMA,PER,PERU,0,557709.7400,22,617856.3300,80900.8300,0,1,55868.0000,1,230971.6300,230971.6300,230971.6300,1
2,202309,6992772,15,E,181,7010.0000,ACTIVIDS.INMOB.C/V.ALQUILER EXPLOTACION,0,0,0,0,0,0,NaN,0,0,Q1N,PEQ. EMPRESA NORMAL,PQ,PEQ. EMPRESA,D,150131,1327,SAN ISIDRO-LIMA,M7,LIMA,30.0000,LIMA,PER,PERU,0,990.0000,1,0.0000,0.0000,0,1,990.0000,1,990.0000,990.0000,990.0000,1
3,202309,7003035,15,E,180,5520.0000,"RESTAURANTES, BARES Y CANTINAS",0,0,0,0,0,0,NaN,0,1,C1N,NEGOCIOS NORMAL,PC,NEGOCIOS,D,150122,1318,MIRAFLORES-LIMA,M7,LIMA,30.0000,LIMA,PER,PERU,0,8201.0800,1,9421.1400,4185.9000,0,0,0.0000,1,8201.0800,8201.0800,8201.0800,1
4,202309,313759,33,E,404,6301.0000,MANIPULACION DE LA CARGA,0,0,0,1,1,0,NaN,1,8,C1N,NEGOCIOS NORMAL,PC,NEGOCIOS,D,150136,1332,SAN MIGUEL-LIMA,M7,LIMA,30.0000,LIMA,PER,PERU,0,81179.8400,22,93680.8600,30669.6200,0,2,19197.0000,2,430.0000,215.0000,667.4100,3


# 3. Escalón 2 - Población con riesgo

In [3]:
esc2=dat[(dat.MTO_INGRESOS_MES>=1100)&(dat.MTO_INGRESOS_MES<=30e+6)]

# 4. Escalón 3 - Directo a evaluación

In [4]:
esc3=dat[dat.MTO_INGRESOS_MES>30e+6]

# 5. Modelamiento

## Modelo 1: k-MEANS

#### Con las variables originales

In [5]:
x_model=['FLG_PERFIL_INGRESOS_3DS', 'MTO_CONOTROSPROXIMOS', 'CTD_CONOTROSPROXIMOS']

In [6]:
train_jur_std = StandardScaler().fit_transform(esc2[x_model])
k = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, max_iter=300,init='k-means++', random_state = 7).fit(train_jur_std)
        scoreSilhoutte = metrics.silhouette_score(train_jur_std, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

 k= 2  SC= 0.8562176355840614
 k= 3  SC= 0.8715785260944452
 k= 4  SC= 0.8832005524929184
 k= 5  SC= 0.8683026114647298
 k= 6  SC= 0.8892477211619237
 k= 7  SC= 0.8547177571868371
 k= 8  SC= 0.8579519812346904
 k= 9  SC= 0.8731149419550169
 k= 10  SC= 0.8703964491247295
 k= 11  SC= 0.8740727254800882
 k= 12  SC= 0.8733082633941185
 k= 13  SC= 0.8683019283491161
 k= 14  SC= 0.88063333886107
 k= 15  SC= 0.8743398802091756
 k= 16  SC= 0.8798966396049042
 k= 17  SC= 0.8838329782107133
 k= 18  SC= 0.8643407545614128
 k= 19  SC= 0.8551303847280429
 k= 20  SC= 0.8561859753346216


#### Reemplazando var mto_conotrosproximos con mto_ingresos_mes

In [7]:
x_model_2=['FLG_PERFIL_INGRESOS_3DS','CTD_CONOTROSPROXIMOS','MTO_INGRESOS_MES']

In [8]:
train_jur_std2 = StandardScaler().fit_transform(esc2[x_model_2])
k = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for ki in k:
        outputKMeans = KMeans(n_clusters = ki, max_iter=300,init='k-means++', random_state = 7).fit(train_jur_std2)
        scoreSilhoutte = metrics.silhouette_score(train_jur_std2, outputKMeans.labels_, metric='euclidean')  
        print(" k=", ki," SC=", scoreSilhoutte)

 k= 2  SC= 0.8508017106914852
 k= 3  SC= 0.871812741822733
 k= 4  SC= 0.8802151270134532
 k= 5  SC= 0.8658067510416491
 k= 6  SC= 0.8393202942381356
 k= 7  SC= 0.852619768783403
 k= 8  SC= 0.8571383117223987
 k= 9  SC= 0.8685811917846823
 k= 10  SC= 0.8681924206596207
 k= 11  SC= 0.8729354015482129
 k= 12  SC= 0.87520140619992
 k= 13  SC= 0.867902453251738
 k= 14  SC= 0.8729426455413386
 k= 15  SC= 0.8771091472708904
 k= 16  SC= 0.8783741258386751
 k= 17  SC= 0.8779942294253453
 k= 18  SC= 0.8771321730057384
 k= 19  SC= 0.8770053545422396
 k= 20  SC= 0.8797769339967209


## Modelo 2: Isolation Forest

#### Con las variables originales

In [9]:
# Evaluación del mejor número de árboles

In [10]:
from matplotlib.pyplot import step
silhouette_list = []

for est in np.arange(100,800,100):
    cluster_labels = IsolationForest(n_estimators = est,
                                    n_jobs=-1,
                                    max_features=(len(esc2[x_model].columns)),
                                    contamination=0.01,
                                    random_state=66,bootstrap=True).fit_predict(esc2[x_model])
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(esc2[x_model],cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(est)+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 100 with n=311 outliers is: 0.9507554197433552
silhouette for 200 with n=315 outliers is: 0.9565559715696512
silhouette for 300 with n=315 outliers is: 0.9570006047568044
silhouette for 400 with n=315 outliers is: 0.9574545905826833
silhouette for 500 with n=315 outliers is: 0.9557953675870137
silhouette for 600 with n=315 outliers is: 0.9561272398175094
silhouette for 700 with n=314 outliers is: 0.9566901502080292


In [11]:
from matplotlib.pyplot import step
silhouette_list = []

for cont in np.arange(0.001,0.011,0.001):
    cluster_labels = IsolationForest(n_estimators = 400,
                                    n_jobs=-1,
                                    max_features=(len(esc2[x_model].columns)),
                                    contamination=cont,
                                    random_state=66,bootstrap=True).fit_predict(esc2[x_model])
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(esc2[x_model],cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 0.001 with n=32 outliers is: 0.9694774475642106
silhouette for 0.002 with n=62 outliers is: 0.9746495982415879
silhouette for 0.003 with n=95 outliers is: 0.97397642809338
silhouette for 0.004 with n=126 outliers is: 0.9695198593289093
silhouette for 0.005 with n=158 outliers is: 0.9686650177929972
silhouette for 0.006 with n=189 outliers is: 0.9660689829757372
silhouette for 0.007 with n=221 outliers is: 0.9633931630319087
silhouette for 0.008 with n=251 outliers is: 0.9608550962212898
silhouette for 0.009 with n=284 outliers is: 0.9597356067271885
silhouette for 0.01 with n=315 outliers is: 0.9574545905826833


#### Reemplazando var mto_conotrosproximos con mto_ingresos_mes

In [12]:
x_model_2=['FLG_PERFIL_INGRESOS_3DS',
 'CTD_CONOTROSPROXIMOS',
 'MTO_INGRESOS_MES']

In [13]:
from matplotlib.pyplot import step
silhouette_list = []

for cont in np.arange(0.001,0.006,0.001):
    cluster_labels = IsolationForest(n_estimators = 400,
                                    n_jobs=-1,
                                    max_features=(len(esc2[x_model_2].columns)),
                                    contamination=cont,
                                    random_state=66,bootstrap=True).fit_predict(esc2[x_model_2])
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(esc2[x_model_2],cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 0.001 with n=32 outliers is: 0.9618419758656127
silhouette for 0.002 with n=63 outliers is: 0.9579390201612932
silhouette for 0.003 with n=95 outliers is: 0.9619953602272171
silhouette for 0.004 with n=126 outliers is: 0.9612048645469289
silhouette for 0.005 with n=158 outliers is: 0.9618791616752965


#### Normalizando

In [14]:
from matplotlib.pyplot import step

df2 = esc2[x_model]

scaler = MinMaxScaler()
df2[x_model] = scaler.fit_transform(df2)
df2=df2[x_model]

silhouette_list = []

for cont in np.arange(0.01,0.15,0.01):
    cluster_labels = IsolationForest(n_estimators = 500,
                                    n_jobs=-1,
                                    max_features=(len(df2.columns)),
                                    contamination=cont,
                                    random_state=66).fit_predict(df2)
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(df2,cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 0.01 with n=315 outliers is: 0.7707019089861397
silhouette for 0.02 with n=630 outliers is: 0.7541838104274898
silhouette for 0.03 with n=945 outliers is: 0.7563107738227867
silhouette for 0.04 with n=1258 outliers is: 0.7615055575472929
silhouette for 0.05 with n=1562 outliers is: 0.7781982574438019
silhouette for 0.06 with n=1889 outliers is: 0.7610675865034899
silhouette for 0.07 with n=2198 outliers is: 0.7545701662999621
silhouette for 0.08 with n=2515 outliers is: 0.7522038017328374
silhouette for 0.09 with n=2833 outliers is: 0.7498436483329464
silhouette for 0.1 with n=3141 outliers is: 0.7357698638899705
silhouette for 0.11 with n=3460 outliers is: 0.7420999978569178
silhouette for 0.12 with n=3777 outliers is: 0.7640003715845486
silhouette for 0.13 with n=4092 outliers is: 0.7889195882833808
silhouette for 0.14 with n=4407 outliers is: 0.817067838053323


#### Estandarizando

In [15]:
df2 = esc2[x_model]

df2[x_model] = StandardScaler().fit_transform(df2[x_model])
df2=df2[x_model]

silhouette_list = []

for cont in np.arange(0.01,0.1,0.01):
    cluster_labels = IsolationForest(n_estimators = 500,
                                    n_jobs=-1,
                                    max_features=(len(df2.columns)),
                                    contamination=cont,
                                    random_state=66).fit_predict(df2)
    out_n = cluster_labels[cluster_labels==-1].shape[0]
    sil = silhouette_score(df2,cluster_labels, metric='euclidean')
    silhouette_list.append(sil)
    print("silhouette for "+str(np.round(cont,4))+" with n=" +str(out_n)+" outliers is: "+str(sil))

silhouette for 0.01 with n=315 outliers is: 0.8986412769672212
silhouette for 0.02 with n=630 outliers is: 0.866086231831972
silhouette for 0.03 with n=945 outliers is: 0.8413513566755545
silhouette for 0.04 with n=1259 outliers is: 0.8233179319761256
silhouette for 0.05 with n=1563 outliers is: 0.8119874194881287
silhouette for 0.06 with n=1889 outliers is: 0.7985205633288245
silhouette for 0.07 with n=2198 outliers is: 0.7866260671066319
silhouette for 0.08 with n=2516 outliers is: 0.7769450456343521
silhouette for 0.09 with n=2833 outliers is: 0.7681420164500319


# Modelo Final

In [16]:
X_train = esc2[x_model]

In [17]:
iforest = IsolationForest(n_estimators = 400,
                                    n_jobs=-1,
                                    max_features=(len(X_train[x_model].columns)),
                                    contamination=0.002,
                                    random_state=66,bootstrap=True).fit(X_train[x_model])

In [18]:
esc2['label_iforest']=iforest.predict(X_train[x_model])

In [19]:
esc2[esc2.label_iforest==-1].PERIODO.value_counts()

202309    14
202308    11
202307    11
202306    10
202305     8
202304     8
Name: PERIODO, dtype: int64

In [20]:
esc2[esc2.label_iforest==-1].describe().T

,count,mean,std,min,25%,50%,75%,max
PERIODO,62.0000,202306.8226,1.7132,202304.0000,202305.2500,202307.0000,202308.0000,202309.0000
CODCLAVECIC,62.0000,4711953.7258,7214392.4443,14570.0000,54071.0000,303187.0000,6314235.7500,24234217.0000
EDAD,62.0000,26.5161,16.0825,3.0000,16.2500,26.5000,31.0000,91.0000
ANTIGUEDAD,62.0000,293.2903,141.6573,35.0000,199.0000,315.5000,367.7500,636.0000
CODACTECONOMICA,62.0000,3204.7097,2305.6652,113.0000,1320.0000,2732.0000,5121.0000,9309.0000
FLG_ACTECO_NODEF,62.0000,0.0484,0.2163,0.0000,0.0000,0.0000,0.0000,1.0000
FLGNP,62.0000,0.1290,0.3380,0.0000,0.0000,0.0000,0.0000,1.0000
CTDNP,62.0000,0.5161,1.7531,0.0000,0.0000,0.0000,0.0000,10.0000
FLGLSB,62.0000,0.2258,0.4215,0.0000,0.0000,0.0000,0.0000,1.0000
CTDLSB,62.0000,1.5806,3.6465,0.0000,0.0000,0.0000,0.0000,15.0000


In [21]:
#Guardamos el modelo
import pickle
pickle.dump(iforest, open(r"../src/02_models/iforest_model_ppjj", 'wb'))